In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/podcastdata/ground_truth.txt
/kaggle/input/podcastdata/raw_audio_processed.wav
/kaggle/input/models/INTERSPEECH-T-BRNN.pcl
/kaggle/input/mydata/sample_truth.txt
/kaggle/input/mydata/sample_text.txt


In this notebook we will comapre the performance of different Automatic Speech Recognition models on large audio files on the basis of character error rate(CER) and word error rate(WER). The data for evluation is obtained from https://colab.research.google.com/gist/titu1994/a44fffd459236988ee52079ff8be1d2e/long-audio-transcription-citrinet.ipynb?pli=1#scrollTo=rZITgro3DC_v  
Data contents:
a) raw_audio_processed.wav
b) ground_truth.txt

**Evaluation function**

In [2]:
## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
print("Finished installing nemo !")

  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-rfpomn1z/nemo-toolkit_7d960611ef154e099f5e90ed54c7f3ac
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-rfpomn1z/nemo-toolkit_7d960611ef154e099f5e90ed54c7f3ac
     |████████████████████████████████| 809 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 144 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 765 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 790 kB/s  eta 0:00:01
  Using cached pytest_runner-5.3.1-py3-none-any.whl (7.1 kB)
     |████████████████████████████████| 2.9 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 179 kB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB

In [3]:
from nemo.collections.asr.metrics.wer import word_error_rate
def evaluate_model(ground_truth,model_transcript):
    with open(ground_truth, 'r') as f:
        ground_truth_txt = f.readlines()
        ground_truth_txt = [text.replace("\n", "") for text in ground_truth_txt]

    with open(model_transcript, 'r') as f:
        transcription_txt = f.readlines()
        transcription_txt = [text.replace("\n", "") for text in transcription_txt]
    cer = word_error_rate(transcription_txt, ground_truth_txt, use_cer=True)
    wer = word_error_rate(transcription_txt, ground_truth_txt, use_cer=False)
    print("Finished computing metrics !")
    print(f"CER : {cer * 100:0.4f}%")
    print(f"WER : {wer * 100:0.4f}%")

[NeMo W 2021-06-18 08:06:51 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-06-18 08:06:54 nemo_logging:349] /opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
      '"sox" backend is being deprecated. '
    


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /usr/share/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


[NeMo W 2021-06-18 08:06:55 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
# Helper function
import contextlib
import torch
import os
import nemo.collections.asr as nemo_asr
# Helper for torch amp autocast
if torch.cuda.is_available():
    autocast = torch.cuda.amp.autocast
else:
    @contextlib.contextmanager
    def autocast():
        print("AMP was not available, using FP32!")
        yield
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# **1. Citrinet 256 (9.8 M parameters)**

In [ ]:
%%time
citrinet_256_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("stt_en_citrinet_256", map_location=device)
citrinet_256_model = citrinet_256_model.to(device)
audio_path = "/kaggle/input/podcastdata/raw_audio_processed.wav"
transcribed_filepath = f"/kaggle/working/citrinet256.txt"
if os.path.exists(transcribed_filepath):
  print(f"File already exists, delete {transcribed_filepath} manually before re-transcribing the audio !")

else:
    with autocast():
        transcript = citrinet_256_model.transcribe([audio_path], batch_size=1)[0]
with open(transcribed_filepath, 'w', encoding='utf-8') as f:
    f.write(f"{transcript}\n")

In [ ]:
evaluate_model('/kaggle/input/podcastdata/ground_truth.txt','/kaggle/working/citrinet256.txt')

# **2. Citrinet 512 (38 M parameters)**

In [ ]:

citrinet_512_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("stt_en_citrinet_512", map_location=device)
citrinet_512_model = citrinet_512_model.to(device)
audio_path = "/kaggle/input/podcastdata/raw_audio_processed.wav"
transcribed_filepath = f"/kaggle/working/citrinet512.txt"
if os.path.exists(transcribed_filepath):
  print(f"File already exists, delete {transcribed_filepath} manually before re-transcribing the audio !")

else:
    with autocast():
        transcript = citrinet_512_model.transcribe([audio_path], batch_size=1)[0]
with open(transcribed_filepath, 'w', encoding='utf-8') as f:
    f.write(f"{transcript}\n")

In [ ]:
evaluate_model('/kaggle/input/podcastdata/ground_truth.txt','/kaggle/working/citrinet512.txt')

# **Citrinet 1024 (142 M parameters)**

In [5]:

citrinet_1024_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained("stt_en_citrinet_1024", map_location=device)
citrinet_1024_model = citrinet_1024_model.to(device)
audio_path = "/kaggle/input/podcastdata/raw_audio_processed.wav"
transcribed_filepath = f"/kaggle/working/citrinet1024.txt"
if os.path.exists(transcribed_filepath):
  print(f"File already exists, delete {transcribed_filepath} manually before re-transcribing the audio !")

else:
    with autocast():
        transcript = citrinet_1024_model.transcribe([audio_path], batch_size=1)[0]
with open(transcribed_filepath, 'w', encoding='utf-8') as f:
    f.write(f"{transcript}\n")

[NeMo I 2021-06-18 08:06:55 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_citrinet_1024/versions/1.0.0rc1/files/stt_en_citrinet_1024.nemo to /root/.cache/torch/NeMo/NeMo_1.0.2/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo
[NeMo I 2021-06-18 08:07:14 common:676] Instantiating model from pre-trained checkpoint
[NeMo I 2021-06-18 08:07:19 mixins:149] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2021-06-18 08:07:20 modelPT:139] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    use_start_end_token: false
    
[NeMo W 2021-06-18 08:07:20 modelPT:146] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    sample_rate: 16000
    batch_size: 32
    shuffle: false
    use_start_end_token: false
    
[NeMo W 2021-06-18 08:07:20 modelPT:153] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a va

[NeMo I 2021-06-18 08:07:20 features:252] PADDING: 16
[NeMo I 2021-06-18 08:07:20 features:269] STFT using torch
[NeMo I 2021-06-18 08:07:31 modelPT:439] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.2/stt_en_citrinet_1024/86acfaf495a53383369fb6c9c547b8dd/stt_en_citrinet_1024.nemo.


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2021-06-18 08:07:33 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-06-18 08:07:34 nemo_logging:349] /opt/conda/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/aten/src/ATen/native/SpectralOps.cpp:590.)
      normalized, onesided, return_complex)
    


In [ ]:
evaluate_model('/kaggle/input/podcastdata/ground_truth.txt','/kaggle/working/citrinet1024.txt')

# **Transformers (Not Working)**

In [ ]:
! pip install -q transformers
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
!git clone https://huggingface.co/facebook/wav2vec2-large-960h-lv60-self

In [ ]:
#load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
#load any audio file of your choice
speech, rate = librosa.load("/kaggle/input/podcastdata/raw_audio_processed.wav",sr=16000)

input_values = tokenizer(speech, return_tensors = 'pt').input_values
#Store logits (non-normalized predictions)
logits = model(input_values).logits


#Store predicted id's
predicted_ids = torch.argmax(logits, dim =-1)
#decode the audio to generate text
transcriptions = tokenizer.decode(predicted_ids[0])
print(transcriptions)

In [ ]:
sample="winston is one of the most laid-back people i know he is tall and slim with black hair and he always wears a t-shirt and black jeans his jeans have holes in them and his baseball boots are scruffy too he usually sits at the back of the class and he often seems to be asleep however when the exam results are given out he always gets an 'A'  i don't think hes as lazy as he appears to be"

In [ ]:
sentences

# **Punctuation restoration repo1**

In [6]:
!git clone https://github.com/xashru/punctuation-restoration.git

Cloning into 'punctuation-restoration'...
remote: Enumerating objects: 276, done.
remote: Total 276 (delta 0), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (276/276), 10.96 MiB | 15.90 MiB/s, done.
Resolving deltas: 100% (166/166), done.


In [7]:

!pip install -r /kaggle/working/punctuation-restoration/requirements.txt


     |████████████████████████████████| 674 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 10.3 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nemo-toolkit 1.0.2 requires transformers>=4.0.1, but you have transformers 2.11.0 which is incompatible.
allennlp 2.3.0 requires transformers<4.6,>=4.1, but you have transformers 2.11.0 which is incompatible.


In [8]:
! pip install gdown
import gdown
url = 'https://drive.google.com/u/0/uc?id=17BPcnHVhpQlsOTC8LEayIFFJ7WkL00cr' 
output = 'roberta-large-en.pt' 
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=17BPcnHVhpQlsOTC8LEayIFFJ7WkL00cr
To: /kaggle/working/roberta-large-en.pt
1.49GB [00:16, 91.9MB/s]


'roberta-large-en.pt'

In [11]:
%%time
!python /kaggle/working/punctuation-restoration/src/inference.py --pretrained-model=roberta-large --weight-path=roberta-large-en.pt --language=en --in-file=/kaggle/input/mydata/sample_truth.txt --out-file=/kaggle/working/punctuated_sample_repo1.txt


2021-06-18 08:11:22.343198: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Punctuated text
Winston is one of the most laidback people I know. He is tall and slim, with black hair. and he always wears a tshirt and black jeans. His jeans have holes in them, and his baseball boots are scruffy too. He usually sits at the back of the class, and he often seems to be asleep. However, when the exam results are given out, he always gets an 'A'. I don't think he's as lazy as he appears to be. 
CPU times: user 290 ms, sys: 82.6 ms, total: 372 ms
Wall time: 23 s


In [12]:
evaluate_model('/kaggle/input/mydata/sample_truth.txt','punctuated_sample_repo1.txt')

Finished computing metrics !
CER : 1.7632%
WER : 7.5000%


# **Punctuator2**

In [ ]:
!pip install punctuator

In [ ]:
%%time
from punctuator import Punctuator
p = Punctuator('/kaggle/input/models/INTERSPEECH-T-BRNN.pcl')
file = open("citrinet1024.txt")

punctuator2=p.punctuate(file.read())

punctuator2

In [ ]:

with open("citrinet_punctuation_restoration.txt", "w") as text_file:
    text_file.write(punctuator2)

In [ ]:
evaluate_model('/kaggle/input/mydata/sample_truth.txt','punctuated_samplepunctuator2.txt')

# **Keras Punctuator**

In [ ]:
!git clone https://github.com/vackosar/keras-punctuator.git

In [ ]:
!cp /kaggle/input/mydata/sample_text.txt /kaggle/working/
!python /kaggle/working/keras-punctuator/punctuator.py /kaggle/working/sample_text.txt

